# Import Packages

In [ ]:
pip install missingno

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Load Data

In [ ]:
# load the csv file

df = pd.read_csv("swiggy.csv")

df.head()

In [ ]:
# shape of data

rows, cols = df.shape

print("The dataset has {} rows and {} columns".format(rows,cols))

In [ ]:
# datatypes of columns

(
    df.dtypes
    .rename("Data Types")
)

In [ ]:
# info about the data

df.info()

**Corrections**:

1. Age column should be numerical.
2. Ratings should be float.
3. Date and time columns should be date and time.
4. Vehicle conditions seems to be categorical column which is integer encoded.
5. Multiple deliveries should be integer column.
6. Target column should also be numbers.

In [ ]:
# missing values in data

df.isna().sum()

In [ ]:
df.isna().sum().sum()

**There are no missing values in the data**

In [ ]:
# sample 50 rows of data for viewing

df.sample(50)

**NaN values in the data**

Investigate further

# Missing values

Missing values are string type in the data with an extra space.

In [ ]:
df.loc[43317,"Delivery_person_Ratings"]

In [ ]:
# String NaNs in the data

(df == "NaN ").sum().sum()

8515 such NaN's in the data

In [ ]:
# column wise string NaNs

(df == "NaN ").sum()

In [ ]:
# replace string NaN with NaN

df.replace("NaN ",np.NaN)

In [ ]:
# verify missing values in the data

df.replace("NaN ",np.NaN).isna().sum().sum()

**Verified that string NaNs converted to Nan**

In [ ]:
# NaN in the weather column
(
    df.loc[:,"Weatherconditions"]
    .str.replace("conditions ","")
    .replace("NaN",np.NaN)
    .isna().sum()
)

**616 missing values in the weather column**

In [ ]:
# df with all the missing values replace

missing_df = (
    df.replace("NaN ",np.NaN)
    .assign(
        Weatherconditions = lambda df_ : (
            df_['Weatherconditions']
            .str.replace("conditions ","")
            .replace("NaN",np.NaN)
            )
    )
)

missing_df

In [ ]:
# missing values in data

missing_df.isna().sum()

In [ ]:
# total missing values in data

missing_df.isna().sum().sum()

In [ ]:
8515 + 616

## Missing Value Analysis

### Matrix

In [ ]:
# missing values matrix

msno.matrix(missing_df)

### Count of Missing Values

In [ ]:
# bar plot for columns having missing data

msno.bar(missing_df)

### Missing Corr

In [ ]:
# missing data correlation

msno.heatmap(missing_df)

**Observations**

1. The delivery person columns are correlated to each other. This means that missingness in these columns is highly related which means lack of rider data.
2. The time ordered column is also related to the rider which shows that the data might be missing due to some network error where the system was unable to log rider details and time of order.
3. There is a very high correlation between weather patterns and the road traffic information. Be clear that this does not mean that the road traffic and weather columns are correlated. High corr in missingness means that if value in one column is missing, it is a high chance that it will be missing in others as well.
3. Road traffic density also shows correlation to the rider as it might be provided through the phone of rider (need to investigate).


In [ ]:
missing_df[["Weatherconditions","Road_traffic_density"]].isna().sum()

In [ ]:
# prove point of missingness

(
    missing_df[["Weatherconditions","Road_traffic_density"]]
    .isna().all(axis=1)
    .sum()
) /  missing_df[["Weatherconditions","Road_traffic_density"]].isna().sum()

### Dendrogram

In [ ]:
# dendrogram of missingness

msno.dendrogram(missing_df)

In [ ]:
# percentage of rows in the data having missing value

(missing_df.isna().any(axis=1).sum() / missing_df.shape[0]) * 100

**About `9%` of the rows in the data have missing values**

# Basic Data Cleaning

In [ ]:
# column names in data

df.columns.tolist()

In [ ]:
def change_column_names(data: pd.DataFrame):
    return (
        data.rename(str.lower,axis=1)
        .rename({
            "delivery_person_id" : "rider_id",
            "delivery_person_age": "age",
            "delivery_person_ratings": "ratings",
            "delivery_location_latitude": "delivery_latitude",
            "delivery_location_longitude": "delivery_longitude",
            "time_orderd": "order_time",
            "time_order_picked": "order_picked_time",
            "weatherconditions": "weather",
            "road_traffic_density": "traffic",
            "city": "city_type",
            "time_taken(min)": "time_taken"},axis=1)
    )

In [ ]:
# change column names

df = change_column_names(df)

In [ ]:
df.head()

In [ ]:
# check for duplicate rows

df.drop(columns=["id","rider_id"]).duplicated().sum()

**There are no duplicate rows in the data**

## Column Wise Cleaning

### ID

In [ ]:
# unique items in ID column

print(f"The number of unique IDs are {df['id'].nunique()}")

**All the ID values are unique**

This column can be dropped from the data

### Rider Id

In [ ]:
# unique rider ids in the column

df['rider_id'].nunique()

**The data contains 1320 unique riders**

In [ ]:
# rider order count in data

df['rider_id'].value_counts()

In [ ]:
# extract city name out of rider id

(
    df['rider_id']
    .str.split("RES")
    .str.get(0)
    .rename("City_Name")
)

### Age

In [ ]:
# datatype

df['age'].dtype

In [ ]:
# min, mean and max values

df['age'].astype(float).describe()

**Minimum Rider age of 15 is concerning**

Investigate further what type of transport is he using?

In [ ]:
# boxplot of the age column

sns.boxplot(df['age'].astype(float))

In [ ]:
# rows of data where rider age is less than 18(minor)


minors_data = df.loc[df['age'].astype('float') < 18]

minors_data

**Observations**:

1. The star ratings of all the riders is 1.
2. The vehicle condition of these riders is very bad.
3. No weather and traffic conditions available.
4. Age of all these riders is 15 which is below the permissable age to drive a vehicle.
5. Latitude and Longitude values in negative which is not possible. India is situated above the equator so all latitudes should be positive and east of meridian line so longitudes are positive as well.

**Seems like removing this data for now makes more sense then fixing it because a lot of data is missing**

In [ ]:
# rows of minors

minor_index = minors_data.index.tolist()

len(minor_index)

### Ratings

In [ ]:
# datatype

df['ratings'].dtype

In [ ]:
# min, mean and max values

df['ratings'].astype(float).describe()

**6 Star rating is possibly a data error**

In [ ]:
# boxplot

sns.boxplot(df['ratings'].astype(float))

1. **The Minors data have ratings as 1 which seems like an anomaly if we generally consider the ratings column distribution.**

2. **The rating of 6 is also one another anomaly in the data as maximum ratings are 5 and not more than that**
    - Has to be investigated
    - If data is problematic than fixing or removing is an option

In [ ]:
# rows where the star rating is 6

six_star_data = df.loc[df['ratings'] == "6"]

len(six_star_data)

In [ ]:
six_star_data

In [ ]:
six_star_index = six_star_data.index.tolist()

**Investigate further before action**

Data removal seems logical at this stage.

### Location Columns

In [ ]:
location_columns = df.columns[4:8].tolist()

location_columns

In [ ]:
location_subset = df.loc[:,location_columns]
location_subset

**Valid values for lat and long**

 > India lies to the north of the equator between `6° 44′` and `35° 30′` north latitude and `68° 7′` and `97° 25′` east longitude.

In [ ]:
# statistical analysis

location_subset.describe()

In [ ]:
# set the lower bound limits for the lat and long

lower_bound_lat = 6.44
lower_bound_long = 68.70

In [ ]:
# rows of data where latitude and longitude values are below the bounds

df.loc[
    (df['restaurant_latitude'] < lower_bound_lat) |
    (df['restaurant_longitude'] < lower_bound_long) |
    (df['delivery_latitude'] < lower_bound_lat) |
    (df['delivery_longitude'] < lower_bound_long)
].sample(50)

In [ ]:
# number of rows in data where lat long are erroneous

location_subset.loc[
    (location_subset['restaurant_latitude'] < lower_bound_lat) |
    (location_subset['restaurant_longitude'] < lower_bound_long) |
    (location_subset['delivery_latitude'] < lower_bound_lat) |
    (location_subset['delivery_longitude'] < lower_bound_long)
].shape[0]

**There are 4071 rows in the data where the lat long columns are messy**

In [ ]:
# statistical summary of problematic rows where lat long is below the country's geographical limits

location_subset.loc[
    (location_subset['restaurant_latitude'] < lower_bound_lat) |
    (location_subset['restaurant_longitude'] < lower_bound_long) |
    (location_subset['delivery_latitude'] < lower_bound_lat) |
    (location_subset['delivery_longitude'] < lower_bound_long)
].describe()

In [ ]:
# boxplots for all the anomalies

location_subset.loc[
    (location_subset['restaurant_latitude'] < lower_bound_lat) |
    (location_subset['restaurant_longitude'] < lower_bound_long) |
    (location_subset['delivery_latitude'] < lower_bound_lat) |
    (location_subset['delivery_longitude'] < lower_bound_long)
].plot(kind="box")

plt.xticks(rotation=45)

**Negative Values are correct just we have to take absolute**

**While calculating the distances we can assign null values to these erroneous data points and then impute the values using advanced imputation techniques**

In [ ]:
# taking the absolute values

(
    location_subset.abs()
    .plot(kind="box")
)

ax = plt.gca()

ax.set_xticklabels(ax.get_xticklabels(),rotation=45)

In [ ]:
# number of rows after taking absolute values

(
    location_subset.abs()
    .loc[lambda df_:
        (df_['restaurant_latitude'] < lower_bound_lat) |
        (df_['restaurant_longitude'] < lower_bound_long) |
        (df_['delivery_latitude'] < lower_bound_lat) |
        (df_['delivery_longitude'] < lower_bound_long)]
    .shape[0]

)

**`3640` rows in the data that have messy lat long values**

In [ ]:
# lat long values less than 1

location_subset.abs().loc[lambda df_:
                        (df_['restaurant_latitude'] < 1) |
                        (df_['restaurant_longitude'] < 1) |
                        (df_['delivery_latitude'] < 1) |
                        (df_['delivery_longitude'] < 1)]

**All the messy lat long values are less than 1**

In [ ]:
def clean_lat_long(data: pd.DataFrame, threshold=1):
    location_columns = location_subset.columns.tolist()

    return (
        data
        .assign(**{
            col: (
                np.where(data[col] < threshold, np.NaN, data[col].values)
            )
            for col in location_columns
        })
    )

In [ ]:
clean_lat_long(df).isna().sum()

### Order Date

In [ ]:
# check for missing values

df['order_date'].isna().sum()

In [ ]:
# unique values in order date

df['order_date'].unique()

In [ ]:
# date range

order_date = pd.to_datetime(df['order_date'],dayfirst=True)

order_date.max() - order_date.min()

In [ ]:
# min and maximum dates

order_date.agg(["min","max"]).set_axis(["start","end"],axis=0)

In [ ]:
# extract day, day name, month and year

def extract_datetime_features(ser):
    date_col = pd.to_datetime(ser,dayfirst=True)

    return (
        pd.DataFrame(
            {
                "day": date_col.dt.day,
                "month": date_col.dt.month,
                "year": date_col.dt.year,
                "day_of_week": date_col.dt.day_name(),
                "is_weekend": date_col.dt.day_name().isin(["Saturday","Sunday"]).astype(int)
            }
        ))

In [ ]:
extract_datetime_features(df['order_date'])

### Order time and Order picked time

In [ ]:
# extract hour info from data

order_time_hr = pd.to_datetime(df.replace("NaN ",np.NaN)['order_time'],format='mixed').dt.hour

order_time_hr

In [ ]:
def time_of_day(ser):
    time_col = pd.to_datetime(ser,format='mixed').dt.hour

    return(
        np.select(condlist=[(ser.between(6,12,inclusive='left')),
                            (ser.between(12,17,inclusive='left')),
                            (ser.between(17,20,inclusive='left')),
                            (ser.between(20,24,inclusive='left'))],
                  choicelist=["morning","afternoon","evening","night"],
                  default="after_midnight")
    )

In [ ]:
time_subset = df.loc[:,["order_time","order_picked_time"]]
time_subset

In [ ]:
(
    time_subset
    .dtypes
)

In [ ]:
time_subset.columns.tolist()

In [ ]:
# calculate the pickup time

(
    time_subset
    .assign(**{
        col: pd.to_datetime(time_subset[col].replace("NaN ",np.NaN).dropna(),format="mixed")
        for col in time_subset.columns.tolist()}
    )
    .assign(
        pickup_time = lambda x: (x['order_picked_time'] - x['order_time']).dt.seconds / 60,
        order_time_hour = lambda x: x['order_time'].dt.hour,
        order_time_of_day = lambda x: x['order_time_hour'].pipe(time_of_day)
    )
    .drop(columns=["order_time","order_picked_time"])
)

### Weather

In [ ]:
# value counts

df['weather'].value_counts()

In [ ]:
# unique values
df['weather'].unique()

In [ ]:
# remove conditions from values

(
    df['weather']
    .str.replace("conditions ","")
    .unique()
)

### Traffic

In [ ]:
# value counts

df['traffic'].value_counts()

In [ ]:
# unique values
df['traffic'].unique()

In [ ]:
(
    df['traffic']
    .replace("NaN ",np.NaN)
    .str.rstrip()
    .str.lower()
    .unique()
)

### Vehicle Condition

In [ ]:
# unique values in column

np.sort(df['vehicle_condition'].unique())

### Type of order

In [ ]:
# value counts

df['type_of_order'].value_counts()

In [ ]:
# unique values
df['type_of_order'].unique()

In [ ]:
(
    df['type_of_order']
    .str.rstrip()
    .str.lower()
    .unique()
)

### Type of vehicle

In [ ]:
# value counts

df['type_of_vehicle'].value_counts()

In [ ]:
# unique values
df['type_of_vehicle'].unique()

In [ ]:
(
    df['type_of_vehicle']
    .str.rstrip()
    .str.lower()
    .unique()
)

### Multiple Deliveries

In [ ]:
# datatype of multiple deliveries column

df['multiple_deliveries'].dtype

In [ ]:
# unique values in column

df['multiple_deliveries'].unique()

In [ ]:
# make the column as integer

(
    df['multiple_deliveries']
    .replace("NaN ",np.NaN)
    .astype(float)
    .unique()
)

### Festival

In [ ]:
# unique values in column

df['festival'].unique()

In [ ]:
(
    df['festival']
    .replace("NaN ",np.NaN)
    .str.rstrip()
    .str.lower()
    .unique()
)

### City type

In [ ]:
# unique values in city type

df['city_type'].unique()

In [ ]:
(
    df['city_type']
    .replace("NaN ",np.NaN)
    .str.rstrip()
    .str.lower()
    .unique()
)

### Time Taken

In [ ]:
# datatype of time taken

df['time_taken'].dtype

In [ ]:
(
    df['time_taken']
    .str.replace("(min) ","")
    .astype(int)
)

## Perform Cleaning and Feature Building

In [ ]:
df.columns

In [ ]:
def data_cleaning(data: pd.DataFrame):

    return (
        data
        .drop(columns="id")
        .drop(index=minor_index)                                                # Minor riders in data dropped
        .drop(index=six_star_index)                                             # six star rated drivers dropped
        .replace("NaN ",np.NaN)                                                 # missing values in the data
        .assign(
            # city column out of rider id
            city_name = lambda x: x['rider_id'].str.split("RES").str.get(0),
            # convert age to float
            age = lambda x: x['age'].astype(float),
            # convert ratings to float
            ratings = lambda x: x['ratings'].astype(float),
            # absolute values for location based columns
            restaurant_latitude = lambda x: x['restaurant_latitude'].abs(),
            restaurant_longitude = lambda x: x['restaurant_longitude'].abs(),
            delivery_latitude = lambda x: x['delivery_latitude'].abs(),
            delivery_longitude = lambda x: x['delivery_longitude'].abs(),
            # order date to datetime and feature extraction
            order_date = lambda x: pd.to_datetime(x['order_date'],
                                                  dayfirst=True),
            order_day = lambda x: x['order_date'].dt.day,
            order_month = lambda x: x['order_date'].dt.month,
            order_day_of_week = lambda x: x['order_date'].dt.day_name().str.lower(),
            is_weekend = lambda x: (x['order_date']
                                    .dt.day_name()
                                    .isin(["Saturday","Sunday"])
                                    .astype(int)),
            # time based columns
            order_time = lambda x: pd.to_datetime(x['order_time'],
                                                  format='mixed'),
            order_picked_time = lambda x: pd.to_datetime(x['order_picked_time'],
                                                         format='mixed'),
            # time taken to pick order
            pickup_time_minutes = lambda x: (
                                            (x['order_picked_time'] - x['order_time'])
                                            .dt.seconds / 60
                                            ),
            # hour in which order was placed
            order_time_hour = lambda x: x['order_time'].dt.hour,
            # time of the day when order was placed
            order_time_of_day = lambda x: (
                                x['order_time_hour'].pipe(time_of_day)),
            # categorical columns
            weather = lambda x: (
                                x['weather']
                                .str.replace("conditions ","")
                                .str.lower()
                                .replace("nan",np.NaN)),
            traffic = lambda x: x["traffic"].str.rstrip().str.lower(),
            type_of_order = lambda x: x['type_of_order'].str.rstrip().str.lower(),
            type_of_vehicle = lambda x: x['type_of_vehicle'].str.rstrip().str.lower(),
            festival = lambda x: x['festival'].str.rstrip().str.lower(),
            city_type = lambda x: x['city_type'].str.rstrip().str.lower(),
            # multiple deliveries column
            multiple_deliveries = lambda x: x['multiple_deliveries'].astype(float),
            # target column modifications
            time_taken = lambda x: (x['time_taken']
                                    .str.replace("(min) ","")
                                    .astype(int)))
        .drop(columns=["order_time","order_picked_time"])
    )

In [ ]:
df

In [ ]:
data_cleaning(df)

In [ ]:
location_subset.columns.tolist()

In [ ]:
def calculate_haversine_distance(df):
    location_columns = location_subset.columns.tolist()
    lat1 = df[location_columns[0]]
    lon1 = df[location_columns[1]]
    lat2 = df[location_columns[2]]
    lon2 = df[location_columns[3]]

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(
        dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    distance = 6371 * c

    return (
        df.assign(
            distance = distance)
    )

In [ ]:
# add more data cleaning steps

cleaned_data = (
                df.pipe(data_cleaning)
                .pipe(clean_lat_long)
                .pipe(calculate_haversine_distance)
                )

cleaned_data

## Validate Data Cleaning

In [ ]:
# age column

cleaned_data['age'].agg(["min","max"])

In [ ]:
# ratings column

cleaned_data['ratings'].agg(["min","max"])

In [ ]:
# location columns

cleaned_data[location_columns].plot(kind="box")

plt.xticks(rotation=45)

display(cleaned_data[location_columns].describe())

In [ ]:
# values in categorical columns

cat_cols = cleaned_data.select_dtypes(include="object").columns.tolist()

for col in cat_cols:
    print(f"For {col} unique values are: {cleaned_data[col].unique()}",end="\n\n")

In [ ]:
# bar plot of missing values

msno.bar(cleaned_data)

In [ ]:
# matrix of missing values

msno.matrix(cleaned_data)

In [ ]:
# correlation chart of missing values
msno.heatmap(cleaned_data)

# Save Cleaned Data

In [ ]:
# save the cleaned data

cleaned_data.to_csv("cleaned_data.csv",index=False)

In [ ]:
# load the cleaned data

cleaned_data_load = pd.read_csv("cleaned_data.csv")

In [ ]:
# data types of cleaned data

cleaned_data_load.dtypes

In [ ]:
cleaned_data_load